# Assignment 2: Information Extraction

In [26]:
import nltk
import re

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

## Task 1: Named Entity Annotation (10 Marks)

Using the IOB tagging scheme annotate all of the named entities (PERson, LOCation, ORGanisation, TIME) in the following sentence:

*Wayne Rooney is a professional footballer from England who last played for Major League Soccer club D.C. United and will join Derby County in January 2020.*

Wayne (B-PER) Rooney (I-PER) is (O) a (O) professional (O) footballer (O) from (O) England (B-LOC) who (O) last (O) played (O) for (O) Major (O) League (O) Soccer (O) club (O) D.C. (B-ORG) United (I-ORG) and (O) will (O) join (O) Derby (B-LOC) County (I-LOC) in (O) January (B-TIME) 2020 (I-TIME) . (O)  
---



---

### For subsequent tasks in this assignment, you will work with the documents in `football_players.txt` to perform various information extraction tasks.

In [2]:
# Download the text file (uncomment the line below in this cell, if not already downloaded from Blackboard)
# !curl "https://ideone.com/plain/OvwDXZ" > football_players.txt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 Read all the documents from `football_players.txt` into a list called `docs`.

In [27]:
docs = []
# your code goes here
paragraphs_file = open("drive/My Drive/datasets/football_players.txt", encoding="UTF-8")
all_content = paragraphs_file.read() #reading all the content in one step
#using the string methods we split it

# https://stackoverflow.com/questions/53641202/how-to-read-from-text-file-into-array-paragraph-by-paragraph/53641266
delimiter = "\n"
paragraphs = all_content.split(delimiter)

for para in paragraphs:
    if len(para)!=0: # checking for non empty paragraphs
        docs.append(para)

## Task 2 (10 Marks)
Write a function that takes a document and returns a list of sentences with part-of-speech tags.

Please keep in mind that the expected output is a list within a list as shown below.


Hint: For this task you need to perform three steps:
1. Sentence Segmentation
1. Word Tokenization
1. Part-of-Speech Tagging

In [28]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

def ie_preprocess(document):
  # your code goes here
  sentence = sent_tokenize(document)        # tokenizing whole paragraph in different sentences
  posTagSentences = []
  for sent in sentence:
      words = word_tokenize(sent)           # tokenizing each sentance into words
      posTagSentence = nltk.pos_tag(words)  # part-of-speech taggaing
      posTagSentences.append(posTagSentence)
  return posTagSentences

Run the cell below to verify your result for the second sentence in the first document.
Expected output: 
`[('He', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('forward', 'NN'), ('and', 'CC'), ('serves', 'NNS'), ('as', 'IN'), ('captain', 'NN'), ('for', 'IN'), ('Portugal', 'NNP'), ('.', '.')]`

In [29]:
first_doc = docs[0]
tagged_sentences = ie_preprocess(first_doc)
tagged_sentences[1]

[('He', 'PRP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('forward', 'NN'),
 ('and', 'CC'),
 ('serves', 'NNS'),
 ('as', 'IN'),
 ('captain', 'NN'),
 ('for', 'IN'),
 ('Portugal', 'NNP'),
 ('.', '.')]

## Task 3 (20 Marks)
Write a function that takes a list of tokens with POS tags for a sentence and returns a list of named entities (NE). 

Hint: Use `binary = True` while calling NE chunk function

In [30]:
def find_named_entities(sent):
  # Named entities are definite noun phrases that refer to specific types of individuals, such as organizations, persons, dates, and so on.
  named_entities = []
  # your code goes here
  chunk = nltk.ne_chunk(sent, binary=True)  # nltk's named entity recognizer
  
  for subtree in chunk.subtrees():
      if subtree.label() == 'NE':           # NE stands for named entity
          entity = ""
          for leaf in subtree.leaves():
              entity = entity + leaf[0] + " "
          named_entities.append(entity.strip())
  return named_entities

Run the cell below to verify your result for the first sentence in the first document.
Expected output: `['Cristiano Ronaldo', 'Santos Aveiro', 'ComM', 'GOIH', 'Portuguese', 'Portuguese', 'Spanish', 'Real Madrid', 'Portugal']`

In [31]:
tagged_sentences = ie_preprocess(docs[0])
find_named_entities(tagged_sentences[0])

['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

## Task 4 (5 Marks)

Implement the `find_all_named_entities` function below to find **all** NEs in a given document.

Hint: Use `find_named_entities` implemented above for this task.

In [32]:
def find_all_named_entities(doc):
    named_entities = []
    # your code goes here
    posTagSentences = ie_preprocess(doc)
    for sentence in posTagSentences:
        named_entity = find_named_entities(sentence)
        if len(named_entity)!= 0:
              named_entities.append(named_entity)  
    # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
    return [item for sublist in named_entities for item in sublist] # return a flatlist and not list of lists       

How many named entities did you find in the first document?

In [33]:
# your code goes here
NE1 = find_all_named_entities(docs[0])
answer = len(set(NE1))
print("Number of unique named enities in first document : ",answer)

Number of unique named enities in first document :  30


## Task 5 (5 Marks)

Find named entities across **all** documents in `football_players.txt`, and save the result into a single flat list.

In [34]:
all_named_entities = []
# your code goes here
for doc in docs:
  named_entity_doc = find_all_named_entities(doc) # named entity of single doc
  all_named_entities.append(named_entity_doc)     # apeending named entities of each doc in single list

# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
all_named_entities = [item for sublist in all_named_entities for item in sublist] 

How many named entities did you find across all documents?

In [35]:
# your code goes here
answer_unique = len(set(all_named_entities))
answer = len(all_named_entities)
print("Number of times named enities existed across all documents : ",answer)
print("Number of unique named enities across all documents : ",answer_unique)

Number of times named enities existed across all documents :  369
Number of unique named enities across all documents :  150


## Task 6 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the `re.compile()` function to create the extraction patterns.

Reference: https://docs.python.org/3/howto/regex.html

**Date of Birth of player**

In [36]:
def date_of_birth(doc):    
    #  born 5 February 1992), : We need to find something in this format
    sentence = sent_tokenize(doc)[0]
    # in regex + represent 1,2,3 ...  matching 
    # \s Matches any whitespace character
    # \w Matches any alphanumeric character [a-zA-Z0-9_]
    # \d Matches any decimal digit [0-9]
    # Reference : https://docs.python.org/3/howto/regex.html
    reg = re.compile(r'born\s\d+\s\w+\s\d+') 
    dob = reg.findall(sentence)[0]
    return dob[5:] # removing 'born '


for i in range(10):
    print(f"DOB of player in doc {i} : ",date_of_birth(docs[i]))

DOB of player in doc 0 :  5 February 1985
DOB of player in doc 1 :  24 June 1987
DOB of player in doc 2 :  5 February 1992
DOB of player in doc 3 :  21 March 1980
DOB of player in doc 4 :  24 October 1985
DOB of player in doc 5 :  3 October 1981
DOB of player in doc 6 :  2 May 1975
DOB of player in doc 7 :  15 October 1988
DOB of player in doc 8 :  16 July 1989
DOB of player in doc 9 :  11 May 1984


Execute the cell below to verify the `date_of_birth` function for the third player. Expected output `5 February 1992`


In [37]:
date_of_birth(docs[2])

'5 February 1992'

**Name of Player**

In [38]:
def name_of_the_player(doc):
  # your code goes here
  pos_sents = ie_preprocess(doc)
  named_entities = []
  # your code goes here
  chunk = nltk.ne_chunk(pos_sents[0]) # nltk's named entity recognizer
  fullName = None

  for subtree in chunk.subtrees():
      if subtree.label() == 'PERSON': # trying to find NE with person tag
          entity = ""
          for leaf in subtree.leaves():
              entity = entity + leaf[0] + " "
          named_entities.append(entity.strip())
  if len(named_entities)!=0:
    fullName = " ".join(named_entities)
  return fullName

# Nltk is not able to find player's name in 6th document / Also sonetime not able to find full name of players
for i in range(10):
    print(f"Name of player {i} : ",name_of_the_player(docs[i]))

Name of player 0 :  Cristiano Santos Aveiro
Name of player 1 :  Lionel
Name of player 2 :  Silva Santos JÃºnior Neymar Neymar Jr.
Name of player 3 :  Ronaldo Assis Moreira
Name of player 4 :  Wayne Mark Rooney
Name of player 5 :  None
Name of player 6 :  David Robert Joseph Beckham
Name of player 7 :  Mesut
Name of player 8 :  Gareth Frank Bale
Name of player 9 :  Iniesta


**Origin country of Player**

In [39]:
def country_of_origin(doc):
  # your code goes here
  pos_sents = ie_preprocess(doc)
  # https://sportytell.com/soccer/best-soccer-countries-world-rankings/
  # These are top 20 country who play football, later new countries can be added accordingly
  # This list is too check if our probable answer/location in in this list, it can be country of origin of that player
  football_countries = ['Brazil','Argentina','France','Germany','England','Spain','Portugal','Uruguay','Belgium','Croatia'
                        'Netherlands','Colombia','Italy','Mexico','Chile','Sweden','Switzerland','German','Denmark','Senegal','Algeria','Welsh']
  named_entities = []
  # your code goes here
  chunk = nltk.ne_chunk(pos_sents[0]) # nltk's named entity recognizer
  originCountry = None
  
  for subtree in chunk.subtrees():
      if subtree.label() == 'GPE': # Trying to find location using nltk namd entities
          entity = ""
          for leaf in subtree.leaves():
              entity = entity + leaf[0] + " "
          named_entities.append(entity.strip())

  # if anything found is in football_countries assign that as country  
  for country in football_countries:
      if country in named_entities:
           originCountry = country
      
  # If we are still not able to find country - try searching near 'national team'
  if originCountry == None:
      # Reference : https://docs.python.org/3/howto/regex.html
      match = re.compile(r'(\w+)\snational team')
      sentences = sent_tokenize(doc)
      for sent in sentences:
          expected_country = match.findall(sent)
          if len(expected_country)!=0:
              country = expected_country[0].strip() # remove spaces
              if country in football_countries:     # if anything found is in football_countries assign that as country
                  originCountry = country
  
  # If we are still not able to find country - try searching near 'footballer'               
  if originCountry == None:
      # Reference : https://docs.python.org/3/howto/regex.html
      match =  re.compile(r'(\w+)\sfootballer')
      sentences = sent_tokenize(doc)
      for sent in sentences:
          expected_country = match.findall(sent)
          if len(expected_country)!=0:
              country = expected_country[0].strip() # remove spaces
              if country in football_countries:     # if anything found is in football_countries assign that as country
                  originCountry = country
  # If we are still not able to find country - try searching near 'professional footballer'
  if originCountry == None:
      # Reference : https://docs.python.org/3/howto/regex.html
      match =  re.compile(r'(\w+)\sprofessional\sfootballer')
      sentences = sent_tokenize(doc)
      for sent in sentences:
          expected_country = match.findall(sent)
          if len(expected_country)!=0:
              country = expected_country[0].strip() # remove spaces
              if country in football_countries:     # if anything found is in football_countries assign that as country
                  originCountry = country
    
  return originCountry
  
 
for i in range(10):
    print(f"Country of player {i} :",country_of_origin(docs[i]))

Country of player 0 : Portugal
Country of player 1 : Argentina
Country of player 2 : Brazil
Country of player 3 : Brazil
Country of player 4 : England
Country of player 5 : Sweden
Country of player 6 : England
Country of player 7 : German
Country of player 8 : Welsh
Country of player 9 : Spain


**Team of player**

In [40]:
def team_of_the_player(doc):
  # your code goes here
  pos_sents = ie_preprocess(doc)
  named_entities = []
  chunk = nltk.ne_chunk(pos_sents[0]) # nltk's named entity recognizer
  
  # some football clubs , more can be added in future
  football_clubs = ['Real Madrid','FC Barcelona','Manchester United','Arsenal','Liverpool','Juventus']
  # https://sportytell.com/soccer/best-soccer-countries-world-rankings/
  # These are top 20 country who play football, later new countries can be added accordingly
  football_countries = ['Brazil','Argentina','France','Germany','England','Spain','Portugal','Uruguay','Belgium','Croatia'
                        'Netherlands','Colombia','Italy','Mexico','Chile','Sweden','Switzerland','Denmark','Senegal','Algeria','Welsh','Wales']
  
  teams = str() # to store team of player
  for subtree in chunk.subtrees():
      if subtree.label() == 'ORGANIZATION': # Trying to find location using nltk named entities organization
          entity = ""
          for leaf in subtree.leaves():
              entity = entity + leaf[0] + " "
          named_entities.append(entity.strip())
          
  for club in football_clubs:
      if club in named_entities:
          teams = teams + str(club)
    
  # we might get some information near 'national team'  - A footballer can have two teams (National team and Club team)               
  match = re.compile(r'(\w+)\snational team')
  sentences = sent_tokenize(doc)
  for sent in sentences:
      expected_country = match.findall(sent)
      if len(expected_country)!=0:
          country = expected_country[0].strip() # remove spaces
          if country in football_countries:     # if anything found is in football_countries assign that as country ( that will also his national team)
              if len(teams)!=0:
                  teams  = teams +" and "+str(country)+ " National Team"
              else:
                  teams = teams + str(country)+ " National Team"  
  return teams


for i in range(10):
    print(f"Team of player {i} :",team_of_the_player(docs[i]))

Team of player 0 : Real Madrid and Portugal National Team
Team of player 1 : FC Barcelona and Argentina National Team
Team of player 2 : FC Barcelona and Brazil National Team
Team of player 3 : FC Barcelona and Brazil National Team
Team of player 4 : England National Team
Team of player 5 : Manchester United and Sweden National Team
Team of player 6 : England National Team
Team of player 7 : Arsenal
Team of player 8 : Real Madrid and Wales National Team
Team of player 9 : FC Barcelona and Spain National Team


**Position of Player in Football game**

In [41]:
def position_of_the_player(doc):
  # your code goes here
  sentences = sent_tokenize(doc)
  # positions possible in the game of football
  pos = ["captain", "striker", "winger", "defensive tackle", "defensive end","left forward","right forward",
         "full back","center back","sweeper","coach", "attacking midfielder","forward", "central midfielder"]
  player_position = []
  for p in pos:
      # search every position possible in football in our document using regex
      for sent in sentences:
          # https://stackoverflow.com/questions/500864/case-insensitive-regular-expression-without-re-compile
          reg = re.compile(r'\b({0})\b'.format(p), re.IGNORECASE)    
          if reg.search(sent) != None:
              player_position.append(p)
  uniquePosition = set(player_position)
  return " and ".join(list(uniquePosition))

for i in range(10):
    print(f"Position of player {i} :",position_of_the_player(docs[i]))

Position of player 0 : forward and captain
Position of player 1 : forward and captain
Position of player 2 : forward
Position of player 3 : forward and attacking midfielder
Position of player 4 : forward
Position of player 5 : striker and captain
Position of player 6 : winger and captain
Position of player 7 : attacking midfielder
Position of player 8 : winger
Position of player 9 : central midfielder and captain


## Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract it.

In [42]:
# your code goes here
def awards_of_player(doc):
    sentences = sent_tokenize(doc)
    # Famous football awards source : Google (We can hardcode as the title of these award almost never changes)
    footbal_awards = ['Ballon d\'Or','FIFA World Player of the Year','European Golden Shoe'
                      ,'UEFA Men\'s Player of the Year','European Golden Shoe','FWA Footballer of the Year'
                      ,'PFA Players\' Player of the Year','Onze d\'Or','FIFA Player of the Century'
                      ,'FIFA World Coach of the Year','FIFA Presidential','FIFA Order of Merit','Globe Soccer'
                      ,'Golden Foot','PSL Award','DZFoot d\'Or','La Liga','FIFPro World 11','PFA Young Player']
    player_awards = []
    # if these words coming in sentences he has probably not won that award
    stop_words = ['runner-up','finalist','nominations','finishing second','represented','appearing','nominated for']
    for sent in sentences:
        for award in footbal_awards:
            X = [ item for item in stop_words if item in sent ] 
            if award in sent and len(X)==0:
                player_awards.append(award)
    return " and ".join(list(set(player_awards)))
    

for i in range(10):
    print(f"Some notable awards of player {i}:",awards_of_player(docs[i]))  

Some notable awards of player 0: Ballon d'Or and FIFA World Player of the Year and European Golden Shoe and La Liga
Some notable awards of player 1: Ballon d'Or and FIFA World Player of the Year and European Golden Shoe and La Liga
Some notable awards of player 2: Ballon d'Or and La Liga
Some notable awards of player 3: Ballon d'Or and FIFA World Player of the Year and La Liga
Some notable awards of player 4: FWA Footballer of the Year and Ballon d'Or and PFA Players' Player of the Year and FIFPro World 11
Some notable awards of player 5: 
Some notable awards of player 6: La Liga
Some notable awards of player 7: La Liga
Some notable awards of player 8: FWA Footballer of the Year and PFA Players' Player of the Year and PFA Young Player
Some notable awards of player 9: 
